# One Way Analysis of ANOVA

Decide if different group have different mean

## Concept: 

- total sum of squares: 
    
    - SStotal = sum((indivival - overall mean) ** 2)

        - Stotal**2 = SStotal / (n - 1) 

        - n is the degree of freedom

    - SStotal = SSexpl + SSunexpl

    - SSbtwn == SSexpl = sum((group mean - overall mean) ** 2)

        - Sb**2 = MSb = SSbtwn / (k - 1)

        - k is the group qty

        - explained, singal
    
    - SSwithin == SSunexpl = sum((indivival - group mean) ** 2)

        - Sw**2 = MSw = SSwithin / (n-k)

        - n is the total observation

        - unexplained, noise

- think about several questions, and try to answer it: 

    - why not everyone have same weight lost? <- this is example shown in the video. 
        
        - diets may different -> explained by diet "X" <- BETWEEN

        - people may different -> unexplained by diet "X" <- WITHIN

    - in the actual life, this could be: 

        - why different engine have different shift? <- unexplained by ...



H0: All means are same (for each group)

H1: At least one diff. 

Assumption: 

- simple random sample

- independent observations

- independent groups

- SD(standard deviation of each group) are same

- each group has large sample size. (> 20). OR  groups are proximately normal

Expect: 

- If Ha true => MSb > MSw => F = MSb/ MSw > 1

- If H0 true => MSb == MSw => F = MSb/ MSw == 1

Wendy's comment (不确定是否正确): 

- about the dfn/ dfd

    - MSb and MSw其实都是从计算sample的variation中转换出来的
    
    - [sample var cal](https://en.wikipedia.org/wiki/Variance)

    - MSw 计算的是各个组中的 sample 的 var; sample var需要 除以 n -1 => 每个组 ni -1 => 需要计算各个组 (ni - 1) * k => n - k

    - MSb 因为是between 组=> n -1 为 k -1 

If the assumption does not meet, like if we have small n, non-normality or need to compare median instead of median

- non-param -> KRUSKAL WALLIS

    - `scipy.stats.kruskal(data_a, data_b, data_c, data_d)`

    - could double check the result, similar as one-way AVONA

- bootstrap/ re-sample approach

In [103]:
# to understand anova

import pandas as pd
from scipy import stats

df = pd.read_csv('DietWeigthLoss.csv')
df['WeightLoss'] = df['WeightLoss\tDiet'].apply(lambda x: float(x.split()[0]))
df['Diet']= df['WeightLoss\tDiet'].apply(lambda x: x.split()[1])
df=df[['WeightLoss', 'Diet']]

wtl = 'WeightLoss'
data_a = df.loc[df['Diet'] == 'A', wtl]
data_b = df.loc[df['Diet'] == 'B', wtl]
data_c = df.loc[df['Diet'] == 'C', wtl]
data_d = df.loc[df['Diet'] == 'D', wtl]

total_data_ls = [data_a, data_b, data_c, data_d]
grp_mean_ls = []
grp_std2_ls = []
for grp in total_data_ls: 
    grp_mean_ls.append(grp.mean())
    grp_std2_ls.append(grp.std() ** 2)

total_mean = df[wtl].mean()

MSb = 0 # variation between group/ explainable/ Mean Square Between
for ind, grpm in enumerate(grp_mean_ls): 
    data_n = len(total_data_ls[ind]) # data number within a group
    MSb += (grpm - total_mean) ** 2 * data_n

grp_num = len(total_data_ls)
total_data_num = df.shape[0]

MSb = MSb / (grp_num - 1)

MSw = 0 # variation within group/ unexplainable/ Mean Square Within
for ind, data in enumerate(total_data_ls): 
    temp = data.apply(lambda x: (x - grp_mean_ls[ind])** 2)
    MSw += temp.sum() 

MSw_1 = 0
for ind, std2 in enumerate(grp_std2_ls): 
    MSw_1 += (len(total_data_ls[ind]) - 1) * std2
MSw = MSw / (total_data_num - grp_num)
print ('MSw is {:.3f}'.format(MSw))
Fstat = MSb/ MSw
print (Fstat)
dfn = grp_num - 1
dfd = total_data_num - grp_num
Pr = 1 - stats.f.cdf(Fstat, dfn, dfd)

print ('P-value = P(Fstat >= 6.1 if H0 is true) = {:.3f}'.format(Pr))
print ('So, accept Ha that at least one differs')

stat, pvalue = stats.f_oneway(data_a, data_b, data_c, data_d)
print (stat, pvalue)

MSw is 5.303
6.117525665200089
P-value = P(Fstat >= 6.1 if H0 is true) = 0.001
So, accept Ha that at least one differs
6.117525665200085 0.001127857509636891


# Multiple Testing

Decide which mean is differnt among the multiple groups

Theory: The probability of making at least one Type I Error (alpha) increases with every additional test we conduct

Which is The more tests we do simultaneously, the greater chance of making a type I error!

## Comparision 

- All Pairwise Comparision: 

    - compare: AB, AC, AD, BC, BC, CD

    - Run independent 2-sample t-test

    - for each test, if we use alpha = .05. => P(type I error) = .05; P(not type I error) = .95

    - for over all tests, P(at least one type I) = 1- P(no type I errors) = 1-(p(no type I error btw A, B) x ... x p(no type I error btw C, D)) = 1- .95 ** 6 = .265 

- TukeyHSD => to check which mean are stats different over others. 

    ```
    from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

    mc = MultiComparison(df[wtl], df['Diet'])

    result = mc.tukeyhsd()
    ```

    - below Bonferroni is just check by hand

- Bonferroni Correction/ Approach

    - alph* = .05/ # comp = .05/ 6 = .00833 => confidence is 99.167% for each of these compare group

    - P(at least one type I) = .00833

    - P(at least one type I /over all) = 1-.99167**6 =4.9%

    - t_val (2.735) <= stats.t.ppf(1-.05/6/2, 15x4 - 4) <= 6 is the group number; 2 is the two tail; 15 * 4 - 4 is the df

    - 5.303 is the MSw get from the ANOVA; 15 is the sample number within a group

    - ![avonva_pair_compare](conf_inter_anova.png)

- Conclusion

    - ![avonva_pair_compare](anova_conclude_btw_grps.png)
    
    - C is significantly over A and B. D is in the middle because D is not significantly over A and B, and also C is not significantly over D

In [110]:
from scipy import stats
import numpy as np

pairwise_cmp = []

t_val_SE = stats.t.ppf(1 - .05/ 6/ 2, 15 * 4 - 4) * ((5.303 * 2/ 15) ** .5)

for i, d in enumerate(total_data_ls): 
    if i == grp_num - 1: break 
    for j in range(i + 1, grp_num): 
        lower = d.mean() - total_data_ls[j].mean() - t_val_SE
        upper = d.mean() - total_data_ls[j].mean() + t_val_SE
        pairwise_cmp.append([lower, upper])


print (np.around(pairwise_cmp, 2))

print ('there is a stats significant different betweem Group A and Group C; Group B and Group C')

array([[-2.03,  2.57],
       [-5.23, -0.63],
       [-3.66,  0.94],
       [-5.51, -0.91],
       [-3.93,  0.67],
       [-0.73,  3.87]])

In [114]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

mc = MultiComparison(df[wtl], df['Diet'])
result = mc.tukeyhsd()

print (result)
print (mc.groupsunique)

stats.kruskal(data_a, data_b, data_c, data_d)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  A      B    -0.2733    -2.5  1.9534 False 
  A      C     2.9333   0.7066  5.16   True 
  A      D      1.36   -0.8667 3.5867 False 
  B      C     3.2067    0.98  5.4334  True 
  B      D     1.6333  -0.5934  3.86  False 
  C      D    -1.5733    -3.8  0.6534 False 
--------------------------------------------
['A' 'B' 'C' 'D']


KruskalResult(statistic=15.902089126163188, pvalue=0.0011876192544820314)